# GPU Credit

Implement a GPU credit calculator.

```python
class GPUCredit:
    # A credit is an offering of GPU balance that expires after some expiration-time.
    # The credit can # be used only during [timestamp, timestamp + expiration].
    # A credit can be repeatedly used until # expiration.
    def addCredit(creditID: str, amount: int, timestamp: int, expiration: int) -> None:
        pass

    # Return the balance remaining on the account at the timestamp,
    # return None if there are no credit left. Note, balance cannot be negative.
    # See edge case below.
    def getBalance(creditId, timestamp: int) -> int | None:
        pass

    def useCredit(creditId: str, timestamp: int, amount: int) -> None:
        pass
```

Example test cases:

Example 1:
```python
gpuCredit = GPUCredit()
gpuCredit.addCredit('microsoft', 10, 10, 30)
gpuCredit.getBalance(0) # returns None
gpuCredit.getBalance(10) # returns 10
gpuCredit.getBalance(40) # returns 10
gpuCredit.getBalance(41) # returns None
```

Example 2:
```python
gpuCredit = GPUCredit()
gpuCredit.addCredit('amazon', 40, 10, 50)
gpuCredit.useCredit(30, 30)
gpuCredit.getBalance(40) # returns 10
gpuCredit.addCredit('google', 20, 60, 10)
gpuCredit.getBalance(60) # returns 30
gpuCredit.getBalance(61) # returns 20
gpuCredit.getBalance(70) # returns 20
gpuCredit.getBalance(71) # returns None
```

Edge Case:
```python
gpuCredit = GPUCredit()
gpuCredit.addCredit('openai', 10, 10, 30)
gpuCredit.useCredit(10, 100000000)
gpuCredit.getBalance(10) # returns None
gpuCredit.addCredit('openai', 10, 20, 10)
gpuCredit.getBalance(20) # returns 10
```

# Intuition

Assume for a sequence of calls the timestamp is not dropping,
keep a list of credits (start, end, amount) and update the list along with calls.

Updates (e.g. `_removeExpiredCredits`) can happen in either calls (addCredit, getBalance, useCredit).
Check with the interviewer which API is frequently called and we can make it more efficient.

In [38]:
import collections

class GPUCredit:
    def __init__(self):
        self.credits = collections.defaultdict(list) # {credit_id: (start, end, amount)}
        self.out_of_balance = collections.defaultdict(bool) # true when the credit is out of balance

    # A credit is an offering of GPU balance that expires after some expiration-time.
    # The credit can # be used only during [timestamp, timestamp + expiration].
    # A credit can be repeatedly used until # expiration.
    def addCredit(self, creditID: str, amount: int, timestamp: int, expiration: int) -> None:
        # self._removeExpiredCredits(creditID, timestamp) # Optional
        self.credits[creditID].append((timestamp, timestamp + expiration, amount))
        self.out_of_balance[creditID] = False

    # Return the balance remaining on the account at the timestamp,
    # return None if there are no credit left. Note, balance cannot be negative.
    # See edge case below.
    def getBalance(self, creditId, timestamp: int) -> int | None:
        if self.out_of_balance[creditId]:
            return None
        # self._removeExpiredCredits(creditId, timestamp) # Optional
        balance = sum(amount for start, end, amount in self.credits[creditId]
                      if start <= timestamp <= end) 
        return balance if balance > 0 else None

    def useCredit(self, creditId: str, timestamp: int, amount: int) -> None:
        # self._removeExpiredCredits(creditId, timestamp) # Optional
        balance = self.getBalance(creditId, timestamp)
        if balance is None or balance < amount:
            self.credits[creditId].clear()
            self.out_of_balance[creditId] = True
            return
        
        # Enough balance, use the credit sorted by expiration time
        credits_by_expiration = sorted(self.credits[creditId], key=lambda x: x[1])
        remaining_amount = amount
        for i in range(len(credits_by_expiration)):
            start, end, credit_amount = credits_by_expiration[i]
            if start <= timestamp <= end:
                if credit_amount >= remaining_amount:
                    credits_by_expiration[i] = (start, end, credit_amount - remaining_amount)
                    break
                else:
                    remaining_amount -= credit_amount
                    credits_by_expiration[i] = (start, end, 0)
        self.credits[creditId] = [(start, end, amount) for start, end, amount
                                  in credits_by_expiration if amount > 0]

    # Optionally called to inprove performance.
    def _removeExpiredCredits(self, creditID: str, timestamp: int) -> None:
        non_expired_credits = [c for c in self.credits[creditID] if c[1] >= timestamp]
        self.credits[creditID] = non_expired_credits

In [39]:
# Example 1
gpuCredit = GPUCredit()
gpuCredit.addCredit('microsoft', 10, 10, 30)
assert(gpuCredit.getBalance('microsoft', 0) is None)  # returns None
assert(gpuCredit.getBalance('microsoft', 10) == 10)   # returns 10
assert(gpuCredit.getBalance('microsoft', 40) == 10)    # returns 10
assert(gpuCredit.getBalance('microsoft', 41) is None) # returns None

# Example 2
gpuCredit = GPUCredit()
gpuCredit.addCredit('amazon', 40, 10, 50)
gpuCredit.useCredit('amazon', 30, 30)
assert(gpuCredit.getBalance('amazon', 40) == 10)   # returns 10
gpuCredit.addCredit('amazon', 20, 60, 10)
assert(gpuCredit.getBalance('amazon', 60) == 30)   # returns 30
assert(gpuCredit.getBalance('amazon', 61) == 20)   # returns 20
assert(gpuCredit.getBalance('amazon', 70) == 20)   # returns 20
assert(gpuCredit.getBalance('amazon', 71) is None) # returns None

# Edge Case
gpuCredit = GPUCredit()
gpuCredit.addCredit('openai', 10, 10, 30)
gpuCredit.useCredit('openai', 10, 100000000)
assert(gpuCredit.getBalance('openai', 10) is None) # returns None
gpuCredit.addCredit('openai', 10, 20, 10)
assert(gpuCredit.getBalance('openai', 20) == 10)   # returns 10

print("All tests passed!")

All tests passed!
